In [2]:
from data_loader import load_data
import numpy as np
import math
import random

In [3]:
def greedy_solution(n_depots, n_customers, capacities, setup_costs, demands, costs):
    customer_assignment = [-1] * n_customers
    remaining_capacity = capacities.copy()
    open_depots = set()

    for c in range(n_customers):
        min_cost = float('inf')
        best_depot = -1
        for d in range(n_depots):
            if remaining_capacity[d] >= demands[c] and costs[c][d] < min_cost:
                min_cost = costs[c][d]
                best_depot = d
        if best_depot == -1:
            raise ValueError("No feasible assignment found")
        customer_assignment[c] = best_depot
        remaining_capacity[best_depot] -= demands[c]
        open_depots.add(best_depot)

    total_cost = sum(setup_costs[d] for d in open_depots)
    total_cost += sum(costs[c][customer_assignment[c]] for c in range(n_customers))
    return list(open_depots), customer_assignment, total_cost

def calculate_cost(open_depots, assignment, setup_costs, costs):
    setup = sum(setup_costs[d] for d in open_depots)
    transport = sum(costs[i][assignment[i]] for i in range(len(assignment)))
    return setup + transport

def simulated_annealing(n_depots, n_customers, capacities, setup_costs, demands, costs,
                        initial_temp=1000, final_temp=1, alpha=0.95, max_iter=1000):
    open_depots, assignment, cost = greedy_solution(n_depots, n_customers, capacities, setup_costs, demands, costs)
    current_open = open_depots.copy()
    current_assignment = assignment.copy()
    current_cost = cost
    best_open = current_open.copy()
    best_assignment = current_assignment.copy()
    best_cost = current_cost

    temp = initial_temp
    while temp > final_temp:
        for _ in range(max_iter):
            new_open = current_open.copy()
            new_assignment = current_assignment.copy()

            # Rastgele bir müşteri seç ve başka bir geçerli depoya ata
            c = random.randint(0, n_customers - 1)
            current_depot = new_assignment[c]
            possible_depots = [d for d in range(n_depots) if d != current_depot and capacities[d] >= demands[c]]
            if not possible_depots:
                continue
            new_depot = random.choice(possible_depots)

            # kapasiteyi güncelle
            depot_capacities = capacities.copy()
            for i in range(n_customers):
                if i != c:
                    depot_capacities[new_assignment[i]] -= demands[i]

            if depot_capacities[new_depot] < demands[c]:
                continue

            new_assignment[c] = new_depot
            new_open = set(new_assignment)
            new_cost = calculate_cost(new_open, new_assignment, setup_costs, costs)

            delta = new_cost - current_cost
            if delta < 0 or random.random() < math.exp(-delta / temp):
                current_open = new_open
                current_assignment = new_assignment
                current_cost = new_cost
                if current_cost < best_cost:
                    best_open = current_open.copy()
                    best_assignment = current_assignment.copy()
                    best_cost = current_cost

        temp *= alpha

    return sorted(list(best_open)), best_assignment, best_cost


In [4]:
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [0, 1, 3, 6, 8, 10, 11, 12, 16, 17, 18, 19, 22, 23, 24]
Müşterilerin Depo Seçimi: [11, 11, 0, 24, 11, 0, 1, 8, 11, 11, 3, 10, 10, 0, 6, 11, 3, 8, 3, 6, 3, 6, 10, 0, 11, 10, 12, 10, 19, 0, 0, 10, 0, 16, 11, 11, 17, 18, 23, 24, 19, 3, 11, 6, 22, 23, 23, 6, 24, 11]
Toplam Maliyet: 801666.68


In [5]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [5, 10, 12, 22, 24, 26, 33, 36, 40, 44, 45]
Müşterilerin Depo Seçimi: [24, 24, 5, 36, 24, 5, 36, 12, 24, 24, 10, 22, 12, 5, 24, 24, 10, 22, 10, 24, 10, 5, 22, 5, 24, 22, 26, 22, 40, 5, 5, 22, 5, 33, 24, 24, 36, 12, 45, 36, 40, 10, 24, 40, 44, 45, 45, 24, 36, 24]
Toplam Maliyet: 863132.51


In [6]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [3, 8, 17, 19, 29, 45, 47, 50, 51, 56, 62, 63, 64, 75, 86, 87, 95, 100, 103, 109, 123, 148, 150, 154, 159, 163, 168, 188]
Müşterilerin Depo Seçimi: [103, 95, 50, 56, 51, 64, 50, 100, 64, 17, 19, 150, 51, 47, 8, 47, 62, 159, 148, 95, 45, 63, 150, 17, 168, 75, 64, 148, 148, 56, 87, 63, 150, 17, 100, 86, 150, 188, 86, 3, 63, 109, 29, 47, 75, 168, 8, 123, 150, 3, 63, 17, 103, 188, 154, 163, 17, 50, 123, 100, 154, 17, 87, 17, 45, 148, 148, 148, 95, 17, 168, 45, 150, 62, 159, 51, 56, 19, 63, 3, 100, 3, 51, 64, 63, 51, 109, 109, 150, 159, 63, 56, 51, 56, 63, 19, 86, 19, 64, 123, 188, 109, 148, 163, 17, 63, 109, 95, 103, 56, 63, 8, 95, 45, 86, 62, 50, 109, 45, 109, 188, 56, 51, 163, 154, 103, 163, 188, 95, 123, 95, 188, 29, 109, 159, 50, 8, 63, 63, 63, 86, 17, 3, 150, 148, 103, 50, 154, 3, 19, 95, 47, 3, 50, 56, 154, 8, 159, 150, 3, 45, 8, 47, 63, 75, 86, 86, 109, 87, 56, 64, 109, 154, 63, 29, 87, 123, 150, 56, 150, 64, 75, 87, 64, 87, 3, 51, 168, 29, 168, 87, 64, 154, 159, 10

In [7]:
filename = 'Dataset/wl_300'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [5, 14, 30, 38, 48, 52, 62, 63, 64, 65, 69, 71, 81, 84, 85, 92, 125, 126, 130, 131, 141, 153, 155, 163, 165, 169, 180, 191, 192, 197, 198, 204, 206, 214, 222, 223, 231, 247, 250, 264, 285, 287]
Müşterilerin Depo Seçimi: [180, 223, 125, 204, 287, 264, 155, 64, 214, 81, 287, 85, 264, 84, 81, 285, 153, 264, 206, 163, 155, 38, 65, 14, 206, 250, 62, 191, 65, 92, 153, 222, 191, 14, 81, 65, 130, 192, 92, 69, 131, 131, 214, 48, 180, 223, 250, 131, 52, 65, 264, 71, 285, 64, 81, 52, 14, 191, 206, 222, 71, 126, 30, 250, 92, 141, 191, 125, 231, 62, 191, 204, 192, 204, 130, 69, 62, 231, 153, 214, 125, 30, 30, 206, 180, 30, 197, 52, 5, 197, 247, 223, 5, 231, 153, 125, 180, 69, 84, 130, 5, 250, 287, 38, 285, 231, 155, 192, 38, 125, 223, 165, 85, 5, 163, 165, 63, 198, 247, 169, 62, 285, 264, 197, 84, 231, 204, 38, 153, 250, 264, 192, 130, 285, 222, 231, 180, 85, 62, 204, 65, 65, 250, 92, 214, 48, 5, 223, 63, 231, 131, 153, 287, 223, 214, 30, 71, 192, 14, 204, 223, 141, 130, 92, 69, 38

In [8]:
filename = 'Dataset/wl_500'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)
open_depots, customer_assignment, total_cost = simulated_annealing(
    n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs
)

print("Seçilen Depolar:", open_depots)
print("Müşterilerin Depo Seçimi:", customer_assignment)
print("Toplam Maliyet:", round(total_cost, 2))

Seçilen Depolar: [5, 15, 16, 18, 37, 50, 51, 60, 63, 67, 70, 77, 106, 107, 109, 118, 122, 132, 133, 138, 141, 146, 150, 154, 159, 162, 164, 166, 181, 184, 195, 199, 200, 209, 222, 229, 237, 238, 247, 255, 271, 276, 281, 283, 285, 310, 311, 328, 342, 352, 362, 366, 380, 381, 389, 395, 414, 417, 423, 424, 425, 429, 432, 436, 437, 442, 445, 446, 452, 462, 466, 469, 474, 478, 486]
Müşterilerin Depo Seçimi: [162, 417, 285, 429, 442, 238, 229, 60, 423, 474, 452, 164, 311, 118, 362, 466, 380, 417, 60, 425, 247, 445, 283, 414, 285, 474, 184, 200, 389, 200, 466, 138, 342, 247, 63, 138, 162, 414, 150, 437, 462, 478, 437, 445, 18, 16, 381, 276, 37, 276, 436, 362, 195, 285, 107, 414, 162, 118, 15, 150, 381, 446, 425, 63, 118, 285, 181, 255, 462, 222, 5, 37, 362, 432, 389, 414, 469, 67, 67, 50, 229, 159, 107, 229, 380, 381, 77, 138, 276, 432, 389, 247, 18, 469, 424, 15, 462, 442, 285, 164, 237, 285, 164, 486, 478, 164, 478, 132, 16, 469, 446, 107, 436, 109, 118, 16, 237, 389, 51, 132, 184, 362, 285